In [1]:
%reload_ext autoreload
%autoreload 2
%run startup.py
from trajectory import Joint
from trajectory.test.cpptest import CPPPlanner

# /Users/eric/Documents/proj/tplan/cmake/src/cli


cp = CPPPlanner('/Users/eric/Documents/proj/trajectory/src/cmake-cli-test/cli')

cp.make()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/eric/Documents/proj/trajectory/src/cmake-cli-test/cli'

In [2]:
!pwd


/Users/eric/Documents/proj/tplan/python/pytplan/test


In [18]:
# Write a test file
from pathlib import Path
from random import random, randint
p  =Path('/Users/eric/Documents/proj/trajectory/src/test/')
signs = [1,1,1]

with open(p/'stepper_file_test.txt', 'w') as f:
    for i in range(100):
        for axis in (0,1):
            signs[axis] = -signs[axis] if random() < .1 else signs[axis] # change direction 10% of the time
        x1 = signs[0]*int(random() < .90)*randint(0,10000)
        x2 = signs[1]*int(random() < .90)*randint(0,10000)
        x3 = signs[2]*int(random() < .90)*randint(0,10000)
        
        f.write(f"{x1} {x2} {x3}\n")




In [93]:
##
## Compare Python  blocks to CPP planner blocks
###
from trajectory.test.cpptest import * 
test_dir = '../src/cmake-cli-test/'

# Create the Python blocks
j = Joint(5000, 50_000)
s = Segment(0, j)

blocks = [
    Block(x=1000, v_0=0, v_1=0, joint=j, segment=s).plan(),       # A
    Block(x=1000, v_0=2500, v_1=2500, joint=j, segment=s).plan(), # B
    Block(x=1000, v_0=5000, v_1=5000, joint=j, segment=s).plan(), # C
    Block(x=200, v_0=5000, v_1=5000, joint=j, segment=s).plan(),  # D
    Block(x=1, v_0=5000, v_1=5000, joint=j, segment=s).plan(),    # E
    Block(x=1000, v_0=5000, v_1=0, joint=j, segment=s).plan(),    # F
    Block(x=1000, v_0=0, v_1=5000, joint=j, segment=s).plan(),    # G
    Block(x=400, v_0=5000, v_1=0, joint=j, segment=s).plan(),     # H
    Block(x=400, v_0=0, v_1=5000, joint=j, segment=s).plan(),     # I
    Block(x=150, v_0=5000, v_1=0, joint=j, segment=s).plan(),     # J
    Block(x=150, v_0=0, v_1=5000, joint=j, segment=s).plan()      # K
]
    
blocks = dict([ (k,v) for k,v in zip("ABCDEFGHIJK", blocks)])

# Run the test_planner program to get the c++ planner blocks
tp = TestPlanner(test_dir)
tp.make()
z = tp.run()
cblocks = z['low level block']['output']

n_diffs = 0
for cb in cblocks:
    b = blocks[cb['_tag']].asdict()
    for k, v in cb.items():
        diffs = compare_blocks(cb, b)
        n_diffs += len(diffs)
        if diffs:
            print(cb['_tag'], diffs)
                
if n_diffs == 0:
    print("No differences")

[ 45%] Built target libplanner
Consolidate compiler generated dependencies of target test_planner
[ 50%] Building CXX object test/CMakeFiles/test_planner.dir/test.cpp.o
[ 55%] Building CXX object test/CMakeFiles/test_planner.dir/test_basic.cpp.o
[ 60%] Building CXX object test/CMakeFiles/test_planner.dir/test_block.cpp.o
[ 65%] Building CXX object test/CMakeFiles/test_planner.dir/test_segment.cpp.o
[ 70%] Building CXX object test/CMakeFiles/test_planner.dir/test_planner.cpp.o
[ 75%] Building CXX object test/CMakeFiles/test_planner.dir/test_stepper.cpp.o
[ 80%] Linking CXX executable test_planner
[ 80%] Built target test_planner
[ 90%] Built target planner
[100%] Built target stepper


FileNotFoundError: [Errno 2] No such file or directory: PosixPath('../src/cmake-cli-test/test_planner')

In [ ]:

#
# Compare segments
#
tp = TestPlanner(test_dir)
tp.make()
s1 = tp.load_segment('basic_segment_1')

s2 = Segment(  0, s1.joints, s1.move)
s2.plan()

diffs = compare_seg(s1, s2)
assert len(diffs) == 0, diffs

In [ ]:
tp = TestPlanner(test_dir)
tp.make()
s1 = tp.load_segment('basic_segment_2')

s2 = Segment(  0, s1.joints, s1.move)
s2.plan()

diffs = compare_seg(s1, s2)

assert len(diffs) == 0, diffs

In [ ]:
#
# Compare the python planner to the CPP planner. 
#
LARGE = 1000
small = 1

j = Joint(5000, 50_000)
joints = [j,j]

moves = ([LARGE,small],[small,LARGE])

sl_p, sl_c = cp.compare_planner(joints, moves)
sl_p, sl_c


In [ ]:

moves = [
    [864, -39, 0, 61],
    [0, 0, 747, 594],
    [864, -39, 0, 61]
]

sl_p, sl_c = cp.compare_planner([j,j,j,j], moves, report = True)
sl_p.plot(), sl_c.plot()


In [ ]:
def make_circle(r=5000, n=100):
    from math import sin, cos
   
    moves = []

    r = r
    lx  = 0
    ly = r
    for i in range(0,n+1):
        a = 2*np.pi*(i/n)
        x = sin(a)*r 
        y = cos(a)*r 
        
        moves.append((round(x-lx),round(y-ly)))
        
        lx = x
        ly = y
            
    return moves

moves = make_circle(r=5000, n=100)

sl_p, sl_c = cp.compare_planner([j,j], moves, report = False)
sl_p.plot(), sl_c.plot()


In [ ]:
%reload_ext autoreload
%autoreload 2
%run startup.py
# Serious discontinuities. 
from trajectory import * 
from trajectory.test.cpptest import * 

j = Joint(5_000, 50_000)
joints = [j]*2
sl = SegmentList(joints)   
sl.move([1000,500])
sl.move([500,1000])
sl.move([1000,500])
sl.move([500,1000])

ts = TestStepper('/Users/eric/Documents/proj/trajectory/src/cmake-cli-test/cli')
ts.run_stepper(sl)




In [ ]:
%reload_ext autoreload
%autoreload 2
%run startup.py
# Serious discontinuities. 
from trajectory import * 
from trajectory.test.cpptest import * 

j = Joint(5_000, 50_000)
joints = [j]*2
sl = SegmentList(joints)   
sl.move([1000,500])
sl.move([500,1000])
sl.move([1000,500])
sl.move([500,1000])

ts = TestStepper('/Users/eric/Documents/proj/trajectory/src/cmake-cli-test/cli')
ts.run_stepper(sl)



In [ ]:
%reload_ext autoreload
%autoreload 2
%run startup.py

LARGE = 1000
small = 500

j = Joint(5000, 50_000)
joints = [j,j]

moves = ([LARGE,small],[small,LARGE], [LARGE,small])

cp.make()
l = cp.run_stepper(joints, moves)

# cpp program runs on time period of 5us, so mult by 1/5us to get actual velocity
p = 1/(5/1e6)

df['v'] = df.x.rolling(500).mean() * p
ax=df.plot('t','v', linewidth=4, figsize=(12,4))


sl = SegmentList(joints)   
[sl.move(m) for m in moves]
sl.plot(ax=ax)